In [1]:
# Python 3.6
# Jupyter notebook

# Anaconda build
import pandas as pd
import requests
import io
import getpass

from ipyleaflet import Map, Marker, CircleMarker, Polygon, basemaps
from ipywidgets import Layout
from math import log, pi

iso_lay = Layout(width='1000px', height='640px')

# pip install simple_salesforce
from simple_salesforce import Salesforce

In [2]:
user = 'x'
token = 'x'

In [3]:
sf = Salesforce(username= user, 
                password= getpass.getpass([]), 
                security_token= token)

········


In [4]:
def Report_API_call(reportID):
    path = "https://healthdata.my.salesforce.com/{}?view=d&snip&export=1&enc=UTF-8&xf=csv".format(reportID)
    response = response = requests.get(path, cookies = {'sid' : sf.session_id})
    return pd.read_csv(io.StringIO(response.text), decimal =',')

# 00O0O00000AkXaq
a = Report_API_call('00O0O00000AkXaq')

CHU_char = ['50.3954387','4.3694558']
a['X-connect'] = a['Installation Type'].apply(lambda x: CHU_char if x == 'X-Connect' else False)

index = a[['Account Name','Address Latitude','Address Longitude']]
D_coo = {i:[x,y] for ix,i,x,y in index.itertuples()}
a['Parent_coor'] = a['Parent Account'].apply(lambda x: D_coo[x] if not(pd.isnull(x)) and x in D_coo else False)
a['Multi_coor'] = a['MultiCenterHost Naam'].apply(lambda x: D_coo[x] if not(pd.isnull(x)) and x in D_coo else False)

In [5]:
m = Map(center = [50.5039, 4.4699], 
        zoom=8, 
        layout= iso_lay,
        basemap=basemaps.OpenStreetMap.BlackAndWhite
        # http://leaflet-extras.github.io/leaflet-providers/preview/
        #BlackAndWhite
       )

coo = a[['Address Longitude',
         'Address Latitude',
         'Account Name',
         'Number of users',
         'Main Installation',
         'Parent Account',
         'MultiCenterHost Naam',
         'Parent_coor',
         'Multi_coor',
         'X-connect'
        ]]

for i,long,lat,name,users,main,parent,multi,pc,mc,xc in coo.itertuples():
    
    cl = 'blue'
    if main: 
        cl = 'green'        
        if xc: 
            cl = 'brown'
            
        mark = Marker(location=[lat, long],
                      title = name, 
                      draggable=False, 
                      # opacity=0.15, 
                      visible = False)
        m.add_layer(mark)
            
    if long:
        mark = CircleMarker(location=[lat, long], 
                            radius=int(2.5*(users/pi)**0.5 + 1),
                            color = cl,
                            weight = int(main),
                            fill_color= cl)
        m.add_layer(mark)
  
    if pc:
        polygon = Polygon(
            locations=[[lat, long], pc],
            color="green",
            weight = 1
            )
        m.add_layer(polygon)        
    elif mc:
        polygon = Polygon(
            locations=[[lat, long], mc],
            color="blue",
            weight = 1
            )
        m.add_layer(polygon)
m

A Jupyter Widget

**Circles**

    Size proportional to # users
    Green, Main installations
    Red, X-Connect    
    Blue, Child accounts
        
**Lines**

    Green, Multicenter installation
    Blue, Child accounts

In [6]:
# 00O0O00000AkYJC
b = Report_API_call('00O0O00000AkYJC')
b['Last Modified Date'] = pd.to_datetime(b['Last Modified Date'], format='%d/%m/%Y')

b['codage'] = b['Codage incoming'] == 'OK'
b['ES'] = b['Elasticsearch'] == 'OK'

idx = b.groupby(['Account: Account Name'])['Last Modified Date'].transform(max) == b['Last Modified Date']
b[idx]


c = a.merge(b[idx], 'left', left_on = 'Account Name', right_on = 'Account: Account Name')

c = c.merge(c[['Account Name','ES']], 'left', left_on = 'Parent Account', right_on = 'Account Name', suffixes = ['','par'])
c = c.merge(c[['Account Name','ES']], 'left', left_on = 'MultiCenterHost Naam', right_on = 'Account Name', suffixes = ['','mul'])

c['ES_all'] = c['ES'] ^ c['ESpar'] ^ c['ESmul']

c = c.merge(c[['Account Name','codage']], 'left', left_on = 'Parent Account', right_on = 'Account Name', suffixes = ['','par'])
c = c.merge(c[['Account Name','codage']], 'left', left_on = 'MultiCenterHost Naam', right_on = 'Account Name', suffixes = ['','mul'])

c['codage_all'] = c['codage'] ^ c['codagepar'] ^ c['codagemul']

In [7]:
m = Map(center = [50.5039, 4.4699], 
        zoom=8, 
        layout= iso_lay,
        basemap=basemaps.OpenStreetMap.BlackAndWhite
        # http://leaflet-extras.github.io/leaflet-providers/preview/
        #BlackAndWhite
       )

coo = c[['Address Longitude',
         'Address Latitude',
         'Account Name',
         'Number of users',
         'Main Installation',
         'Parent Account',
         'MultiCenterHost Naam',
         
         'Parent_coor',
         'Multi_coor',
         
         'ES_all',
         'codage_all'
        ]]

for i,long,lat,name,users,main,parent,multi,pc,mc,es,ca in coo.itertuples():
    
    cl = 'green'
    if not(es):
        cl = 'red'
    
    if main:            
        mark = Marker(location=[lat, long],
                      title = name, 
                      draggable=False, 
                      # opacity=0.15, 
                      visible = False)
        m.add_layer(mark)
            
    if long:
        mark = CircleMarker(location=[lat, long], 
                            radius=int(2.5*(users/pi)**0.5 + 1),
                            color = cl,
                            weight = 1,
                            fill_color= cl)
        m.add_layer(mark)
  
    if pc:
        polygon = Polygon(
            locations=[[lat, long], pc],
            color=cl,
            weight = 1
            )
        m.add_layer(polygon)        
    elif mc:
        polygon = Polygon(
            locations=[[lat, long], mc],
            color=cl,
            weight = 1
            )
        m.add_layer(polygon)
m

A Jupyter Widget

In [8]:
m = Map(center = [50.5039, 4.4699], 
        zoom=8, 
        layout= iso_lay,
        basemap=basemaps.OpenStreetMap.BlackAndWhite
        # http://leaflet-extras.github.io/leaflet-providers/preview/
        #BlackAndWhite
       )

coo = c[['Address Longitude',
         'Address Latitude',
         'Account Name',
         'Number of users',
         'Main Installation',
         'Parent Account',
         'MultiCenterHost Naam',
         
         'Parent_coor',
         'Multi_coor',
         
         'ES_all',
         'codage_all'
        ]]

for i,long,lat,name,users,main,parent,multi,pc,mc,es,ca in coo.itertuples():
    
    cl = 'green'
    if not(ca):
        cl = 'red'
    
    if main:            
        mark = Marker(location=[lat, long],
                      title = name, 
                      draggable=False, 
                      # opacity=0.15, 
                      visible = False)
        m.add_layer(mark)
            
    if long:
        mark = CircleMarker(location=[lat, long], 
                            radius=int(2.5*(users/pi)**0.5 + 1),
                            color = cl,
                            weight = 1,
                            fill_color= cl)
        m.add_layer(mark)
  
    if pc:
        polygon = Polygon(
            locations=[[lat, long], pc],
            color=cl,
            weight = 1
            )
        m.add_layer(polygon)        
    elif mc:
        polygon = Polygon(
            locations=[[lat, long], mc],
            color=cl,
            weight = 1
            )
        m.add_layer(polygon)
m

A Jupyter Widget